# Preliminaries for Using the Marlowe Starter Kit

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The [demeter.run](https://demeter.run/) web3 development platform provides an extension *Cardano Marlowe Runtime* that has Marlowe tools installed and makes available the Marlowe Runtime backend services and a Cardano node. If you are not using [demeter.run](https://demeter.run/), then see [the docker page](docker.md) for instructions on deploying Marlowe Runtime using docker.

This notebook provides instructions on setting up signing keys and addresses for this starter kit. It covers the following information:

- Marlowe tools
- Creating addresses and signing keys
    - The faucet
    - The lender
    - The borrower
- Obtaining test ada
- Fund the addresses of the parties
    - Using Daedalus or a web-browser wallet
    - Using a local faucet at the command line
    
[A video works through this Jupyter notebook.](https://youtu.be/hGBmj9ZrYHs)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

## Marlowe Tools

Three alternative workflows are available for running Marlowe contracts:
1. Marlowe CLI (`marlowe-cli`) for lightweight experiments with Marlowe transactions.
2. Marlowe Runtime CLI (`marlowe-runtime-cli`) for non-web applications that use the Marlowe Runtime backend services.
3. Marlowe Runtime Web (`marlowe-web-server`) for web applications that use Marlowe Runtime backend services.

Marlowe Runtime provides a variety of transaction-building, UTxO management, querying, and submission services for using Marlowe contracts: this makes it easy to run Marlowe contracts without attending to the details of the Cardano ledger and Plutus smart contracts. On the contrary, Marlowe CLI does not support querying and UTxO management, so it is best suited for experienced Cardano developers.

![Tools for Running and Querying Marlowe Contracts](images/marlowe-tools.png)

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [1]:
if [[ -z "$MARLOWE_RT_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"

CARDANO_NODE_SOCKET_PATH = /ipc/node.socket
CARDANO_TESTNET_MAGIC = 2


Note the test network magic number:
- `preprod` = 1
- `preview` = 2

## Creating Addresses and Signing Keys

The [Cardano Developers Portal](https://developers.cardano.org/docs/stake-pool-course/handbook/keys-addresses/) contains instructions for creating addresses and signing keys.

This starter kit uses the following addresses:
- An ***optional*** local *Faucet* used to fund parties to Marlowe contracts.
- The *Lender* party for the examples in this starter kit.
- The *Borrower* party for the examples in this starter kit.
- The *Mediator* party for some examples in this starter kit.

The instructions below detail how to create signing keys and addresses for these parties. It is assumed that one has the signing key and address for the faucet and that the faucet is already funded with test ada.

***IMPORTANT:*** The [keys/](keys/) folder holds the signing keys that will be created for interacting with the Marlowe contract. If you delete or lose these files, then you also forever lose the test ada stored at those addresses. Either backup these files or, after running the tutorials, send the remaining test ada back to a more permanent wallet or return it to the faucet.

### The Faucet

***This step is optional if one is using a wallet already funded with test ada.***

Set the file names for this party's signing key and verification key.

In [33]:
FAUCET_SKEY=keys/faucet.skey
FAUCET_VKEY=keys/faucet.vkey

Generate the keys if they haven't already been generated.

In [3]:
if [[ ! -e "$FAUCET_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$FAUCET_SKEY" \
    --verification-key-file "$FAUCET_VKEY"
fi

Compute the faucet's address on the testnet.

In [4]:
FAUCET_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$FAUCET_VKEY" )
echo "$FAUCET_ADDR" > keys/faucet.address
echo "FAUCET_ADDR = $FAUCET_ADDR"

FAUCET_ADDR = addr_test1vz9f3p5p9u8j2w3hq38t8tddpsce7y82ntgn35wxlqjqlts76xv6d


### The Alice

Set the file names for this party's signing key and verification key.

In [41]:
ALICE_SKEY=keys/alice.skey
ALICE_VKEY=keys/alice.vkey

Generate the keys if they haven't already been generated.

In [6]:
if [[ ! -e "$ALICE_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$ALICE_SKEY" \
    --verification-key-file "$ALICE_VKEY"
fi

Compute the party's address on the testnet.

In [43]:
ALICE_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$ALICE_VKEY" )
echo "$ALICE_ADDR" > keys/alice.address
echo "ALICE_ADDR = $ALICE_ADDR"

ALICE_ADDR = addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


### The Bob

Set the file names for this party's signing key and verification key.

In [40]:
BOB_SKEY=keys/bob.skey
BOB_VKEY=keys/bob.vkey

Generate the keys if they haven't already been generated.

In [9]:
if [[ ! -e "$BOB_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$BOB_SKEY"  \
    --verification-key-file "$BOB_VKEY"
fi

Compute the party's address on the testnet.

In [44]:
BOB_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$BOB_VKEY" )
echo "$BOB_ADDR" > keys/bob.address
echo "BOB_ADDR = $BOB_ADDR"

BOB_ADDR = addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


## Obtaining Test Ada

In order to execute transactions on a Cardano network, one needs the native currency ada to pay fees and use as funds. There are the faucets for the public testnets at https://docs.cardano.org/cardano-testnet/tools/faucet where one can obtain test ada daily.

Optionally, it can be convenient to centrally manage funds with the [Daedalus wallet](https://docs.cardano.org/cardano-testnet/daedalus-testnet) or one of the [web-browser wallets](https://builtoncardano.com/ecosystem/wallets): be sure to select the correct public testnet if using one of these wallets.

If you will be using a local faucet, then send test ada to the faucet address created in the previous section. Otherwise, send the test ada to the Daedalus or web-browser wallet.

In [38]:
echo "FAUCET_ADDR = $FAUCET_ADDR"

FAUCET_ADDR = addr_test1vz9f3p5p9u8j2w3hq38t8tddpsce7y82ntgn35wxlqjqlts76xv6d


## Fund the Addresses of the Parties

We'll fund each address with 1000 test ada.

In [45]:
echo "ALICE_ADDR = $ALICE_ADDR"
echo "BOB_ADDR = $BOB_ADDR"

ALICE_ADDR = addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq
BOB_ADDR = addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


### Using a Local Faucet at the Command Line

One can use `cardano-cli` or `marlowe-cli` send funds to an address. Here we use `marlowe-cli`.

***If you have just funded `FAUCET_ADDR` with ada, you may have to wait a couple of minutes before that transaction is confirmed. If the command below fails, then retry it until it succeeds.***

In [50]:
# Note that `FAUCET_ADDR` must have already been funded with test ada.

# 1 ada = 1,000,000 lovelace
ADA=1000000

# Send 1000 ada
AMOUNT=$((1000 * ADA))

# Execute the transaction.
marlowe-cli util fund-address \
 --lovelace "$AMOUNT" \
 --out-file /dev/null \
 --source-wallet-credentials "$FAUCET_ADDR":"$FAUCET_SKEY" \
 --submit 600 \
 "$ALICE_ADDR" "$BOB_ADDR"

TxId "d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c"


See that the addresses have indeed been funded:

In [51]:
echo
echo "Lender @ $LENDER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"
echo

echo
echo "Borrower @ $BORROWER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"
echo


Lender @ addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1b1513e71d14fd6dcb6b90076abcb5548d2492d725f68f42fcd4aad95e1f4354     0        897575765 lovelace + TxOutDatumNone
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
bd58fa8f26eeb65417080126b87a3be56dae66639dde5494bc00bbd7c49278f1     1        1000000000 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     1        1000000000 lovelace + TxOutDatumNone
e941faa76c97cb1956d2e50ac4dc5408e0f92e355ba8e80d480ffa3a2859eb5e     0        816333124 lovelace + TxOutDatumNone


Borrower @ addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf
                           TxHash                                 TxIx        Amount
---------------------